In [ ]:
import sefixlines

## Set **seed**

In [ ]:
sefixlines.data.set_all_seeds()

## Data

### **Initial**

In [ ]:
image_paths = []
mask_paths = []

### **Split**

In [ ]:
from sklearn.model_selection import train_test_split

train_image_paths, valid_image_paths, train_mask_paths, valid_mask_paths = train_test_split(image_paths, mask_paths, test_size=0.2, random_state=42)

### **Augmentations**

In [ ]:
import albumentations as A

sefixlines.data.ImageSemanticSegmentationDataset.augmentation = A.Compose([
    # Добавьте своих аугментаций (по желанию)
])

### Create **Datasets**

In [ ]:
dataset = sefixlines.data.ImageSemanticSegmentationDataset(image_paths, mask_paths)

train_set = sefixlines.data.ImageSemanticSegmentationDataset(train_image_paths, train_mask_paths, True)
valid_set = sefixlines.data.ImageSemanticSegmentationDataset(valid_image_paths, valid_mask_paths)

### Create **DataLoader**

In [ ]:
from torch.utils.data import DataLoader

batch_size = 16
num_workers = 0 # ! Возникают ошибки

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, num_workers=num_workers, shuffle=False)

### ***Visualization***

In [ ]:
sefixlines.data.show_dataset(dataset)

## **Models**

In [ ]:
import segmentation_models_pytorch as smp
from torch import nn, optim
from transformers import AutoModelForImageClassification

In [ ]:
class CustomOutput(nn.Module):
    def __init__(self, model, output_transform=lambda out: out.logits):
        super().__init__()
        self.model = model
        self.output_transform = output_transform

    def forward(self, *args, **kwargs):
        return self.output_transform(self.model(*args, **kwargs))

    def __getattr__(self, name):
        if name in ('model', 'output_transform'):
            return super().__getattr__(name)
        return getattr(self.model, name)
    
    def __setattr__(self, name, value):
        if name in ('model', 'output_transform'):
            super().__setattr__(name, value)
        else:
            setattr(self.model, name, value)

### *Score*

In [ ]:
scores = dict()

### **Model**: `Unet`

In [ ]:
model_image_size = (224, 224)
sefixlines.data.ImageSemanticSegmentationDataset.change_image_size(model_image_size)

In [ ]:
model = smp.Unet(
    in_channels=3, 
    classes=2
)

model_wrapped = sefixlines.models.SemanticSegmenter(model, "Unet")

In [ ]:
model_wrapped.fit(train_loader, valid_loader, num_epochs=3)

In [ ]:
scores[model_wrapped.best_score] = model_wrapped
model_wrapped.visualize_segmentation(valid_set)

## Result

In [ ]:
best_model_wrapped = scores[max(scores)]
best_model_wrapped.name

## Submission

In [ ]:
test_dir = ""

test_image_names = os.listdir(test_dir)
test_image_paths = [f"{test_dir}/{image_name}" for image_name in test_image_names]
test_set = sefixlines.data.ImageSemanticSegmentationDataset(test_image_paths)

In [ ]:
predict_mask = best_model_wrapped.predict(test_set)